### Just trying to understand the data structure and the scoring function.

In [21]:
import pandas as pd
import numpy as np
import os
import torch
from torchinfo import summary

In [20]:
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

In [114]:
# Load index files
base_path = '/kaggle/input/mtcaic3'  # Replace with the path to the dataset directory if needed
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
validation_df = pd.read_csv(os.path.join(base_path, 'validation.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

In [3]:
validation_df[validation_df["task"] == "MI"]["label"].value_counts()

label
Left     28
Right    22
Name: count, dtype: int64

In [4]:
validation_df[validation_df["task"] == "SSVEP"]["label"].value_counts()

label
Backward    14
Left        14
Forward     12
Right       10
Name: count, dtype: int64

In [5]:
train_df[train_df["task"] == "MI"]["label"].value_counts()

label
Right    1213
Left     1187
Name: count, dtype: int64

In [6]:
train_df[train_df["task"] == "SSVEP"]["label"].value_counts()

label
Backward    636
Right       599
Left        585
Forward     580
Name: count, dtype: int64

## Create Label Mapping

In [ ]:
label_mapping = {label: i for i, label in enumerate(train_df["label"].unique())}
label_mapping

{'Left': 0, 'Right': 1, 'Forward': 2, 'Backward': 3}

## Torch Dataset Definition

In [8]:
from torch.utils.data import Dataset, DataLoader


class BCIDataset(Dataset):
    def __init__(self, csv_file, base_path, task_type='MI', label_mapping=None):
        # Filter the main dataframe for the specific task (MI or SSVEP)
        self.metadata = pd.read_csv(os.path.join(base_path, csv_file))
        self.metadata = self.metadata[self.metadata['task'] == task_type]
        self.base_path = base_path
        self.task_type = task_type
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        
        # Determine dataset split (train/validation/test)
        id_num = row['id']
        if id_num <= 4800: 
            dataset_split = 'train'
        elif id_num <= 4900: 
            dataset_split = 'validation'
        else: 
            dataset_split = 'test'
            
        # Path to the EEG data file
        eeg_path = os.path.join(self.base_path, row['task'], dataset_split, 
                                row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
        
        eeg_data = pd.read_csv(eeg_path)
        
        # Extract the correct trial segment
        trial_num = int(row['trial'])
        
        # 9 seconds * 250 Hz = 2250 for MI
        # 7 seconds * 250 Hz = 1750 for SSVEP
        samples_per_trial = 2250 if self.task_type == 'MI' else 1750
        start_idx = (trial_num - 1) * samples_per_trial
        end_idx = start_idx + samples_per_trial
        
        # Select only the 8 EEG channels
        eeg_channels = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
        trial_data = eeg_data.loc[start_idx:end_idx-1, eeg_channels].values
        
        # uncomment the line below and comment the one above to include all 18 columns
        # trial_data = eeg_data.loc[start_idx:end_idx-1].values
        
        # Preprocess the data (see next section)
        processed_data = self.preprocess(trial_data)
        
        # Convert to tensor
        tensor_data = torch.tensor(processed_data, dtype=torch.float32)
        
        # Get label if it exists
        if 'label' in row and self.label_mapping:
            label_str = row['label']
            
            label_int = self.label_mapping[label_str]
            label = torch.tensor(label_int, dtype=torch.long) # Use torch.long for labels in classification
            return tensor_data, label
        else:
            return tensor_data

    def preprocess(self, eeg_data):
        # Apply preprocessing steps here (filtering, normalization, etc.)
        # This will be different for MI and SSVEP
        # ...
        return eeg_data

## MI Model Training

In [15]:
train_mi = BCIDataset('train.csv', base_path, task_type='MI', label_mapping=label_mapping)
val_mi = BCIDataset('validation.csv', base_path, task_type='MI', label_mapping=label_mapping)
test_mi = BCIDataset('test.csv', base_path, task_type='MI', label_mapping=label_mapping)

In [47]:
batch_size = 32

train_loader = DataLoader(train_mi, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_mi, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_mi, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
import torch
from torch import nn, optim


# Simple CNN model
class BCIModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(BCIModel, self).__init__()
        self.conv1 = nn.Conv1d(input_size, 16, kernel_size=3)  # 8 channels
        self.pool = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(16 * 1124, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


In [ ]:
model = BCIModel(8, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data.transpose(1, 2))
        
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, labels in val_loader:
            outputs = model(data.transpose(1, 2))
            
            val_loss += criterion(outputs, labels).item()

    print(f'Epoch {epoch+1}, Val Loss: {val_loss / len(val_loader)}')

In [67]:
summary(model, 
        input_size=(1, 8, 2250),
        col_names=['input_size',
                   'output_size',
                   'num_params'])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
BCIModel                                 [1, 8, 2250]              [1, 2]                    --
├─Conv1d: 1-1                            [1, 8, 2250]              [1, 16, 2248]             400
├─MaxPool1d: 1-2                         [1, 16, 2248]             [1, 16, 1124]             --
├─Linear: 1-3                            [1, 17984]                [1, 2]                    35,970
Total params: 36,370
Trainable params: 36,370
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.94
Input size (MB): 0.07
Forward/backward pass size (MB): 0.29
Params size (MB): 0.15
Estimated Total Size (MB): 0.51

In [57]:
from torcheval.metrics.functional import multiclass_f1_score, binary_accuracy, binary_f1_score

In [68]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for x_batch, y_batch in val_loader:
        logits = model(x_batch.transpose(1, 2))
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds)
        all_labels.append(y_batch)

# Concatenate all predictions and labels
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

f1 = binary_f1_score(all_preds, all_labels)
print(f"F1 Score: {f1.item():.4f}")

multi_f1_micro = multiclass_f1_score(all_preds, all_labels, num_classes=2, average='micro')
print(f"Multiclass F1 Score (micro): {f1.item():.4f}")

multi_f1_macro = multiclass_f1_score(all_preds, all_labels, num_classes=2, average='macro')
print(f"Multiclass F1 Score (macro): {f1.item():.4f}")

acc = binary_accuracy(all_preds, all_labels)
print(f"Accuracy: {acc.item():.4f}")

F1 Score: 0.6111
Multiclass F1 Score (micro): 0.6111
Multiclass F1 Score (macro): 0.6111
Accuracy: 0.4400
